In [1]:
import pandas as pd
pubmed = pd.read_csv('data/all_pubmed.csv')
pubmed.head()

,pubmed_id,title,keywords,journal,abstract,methods,results,conclusions,publication_date,category
0,40073121,Targeting the NPY/NPY1R signaling axis in muta...,NaN,Science advances,Pancreatic cancer (PC) is a highly metastatic ...,NaN,NaN,NaN,2025-03-12,Pubmed_Pancreatic_Cancer.csv
1,40069621,The value of preoperative RDW for post-pancrea...,"Pancreatic ductal adenocarcinoma, Post-pancrea...",BMC cancer,Pancreatic ductal adenocarcinoma (PDAC) is a h...,NaN,A total of 2268 patients were analyzed. We fou...,The preoperative RDW may be a useful marker fo...,2025-03-12,Pubmed_Pancreatic_Cancer.csv
2,40069616,"Protocol of the IMPACT study: randomized, mult...","Atezolizumab, Bevacizumab, Conversion, Hepatoc...",BMC cancer,Atezolizumab plus bevacizumab is recommended a...,NaN,NaN,NaN,2025-03-12,Pubmed_Pancreatic_Cancer.csv
3,40066089,Association between human leukocyte antigen E ...,"HLA-E, cancer, human leukocyte antigen, immuno...",Frontiers in oncology,Immunotherapy has gained momentum with the dis...,NaN,"After screening 657 articles, 11 studies were ...",This systematic review highlights that HLA-E e...,2025-03-11,Pubmed_Pancreatic_Cancer.csv
4,40065459,Oncological and Survival Endpoints in Cancer C...,"adverse events, cachexia, cancer, clinical tri...","Journal of cachexia, sarcopenia and muscle","In patients receiving anti-cancer treatment, c...",NaN,"Fifty-seven trials were eligible, totalling 97...","In CC trials, oncological endpoints were mostl...",2025-03-11,Pubmed_Pancreatic_Cancer.csv


In [2]:
import pandas as pd
import re
import json

# Function to extract NCT IDs safely
def extract_nct_ids_from_abstract(abstract):
    """
    Extract all NCT IDs from the abstract using regex.
    Ensure that the input is a string before applying regex.
    """
    if not isinstance(abstract, str):  # Ensure it's a valid string
        return []
    return re.findall(r'NCT\d+', abstract)

# Function to link PubMed articles to Clinical Trials
def link_pubmed_to_trials(pubmed_df):
    """
    Link PubMed articles to Clinical Trials using NCT IDs extracted from the abstracts.
    """
    data = []
    
    for index, row in pubmed_df.iterrows():
        pubmed_id = row['pubmed_id']
        abstract = row.get('abstract', '')  # Get abstract safely, default to empty string
        nct_ids = extract_nct_ids_from_abstract(abstract)
        
        if nct_ids:  # Only add if there are valid NCT IDs
            data.append({"PubMed_ID": pubmed_id, "NCT_IDs": nct_ids})
    
    return data

# Extracting and saving to JSON
linked_data = link_pubmed_to_trials(pubmed)

# Saving results to JSON
output_file = "linked_pubmed_nct_ids.json"
with open(output_file, 'w') as json_file:
    json.dump(linked_data, json_file, indent=4)

print(f"Data saved in {output_file}")


Data saved in linked_pubmed_nct_ids.json


In [3]:
import pandas as pd
import json

# pd.read_csv('data/all_diseas_processed.csv')
all_disease = pd.read_csv('data/all_disease_procesed.csv')
print(all_disease.head())

pubmed_links = json.load(open('linked_pubmed_nct_ids.json'))
print(pubmed_links[:5])

        NCT ID  Acronym           Overall Status  Start Date  \
0  NCT02695966     EVIS                COMPLETED     2015-05   
1  NCT03882866  Unknown                  UNKNOWN  2019-03-28   
2  NCT04572165  Unknown  ENROLLING_BY_INVITATION  2021-01-26   
3  NCT03190265  Unknown                COMPLETED  2017-12-14   
4  NCT02822066  Unknown                  UNKNOWN     2016-04   

                  Conditions  \
0          Pancreatic Cancer   
1          Pancreatic Cancer   
2  Diabetes Mellitus, Type 2   
3          Pancreatic Cancer   
4       Refractory Neoplasms   

                                       Interventions  \
0                                   Whipples Surgery   
1  3D-printed non-coplanar template, 3D-printed c...   
2             Ozempic®, Active Comparator, Rybelsus®   
3  Cyclophosphamide, Nivolumab, Ipilimumab, GVAX ...   
4                                    Electroporation   

                                           Locations Primary Completion Date  \
0    

In [8]:
# Create a mapping of NCT ID to PubMed ID
nct_to_pubmed = {}

for entry in pubmed_links:
    pubmed_id = str(entry["PubMed_ID"])  # Ensure string format
    for nct_id in entry["NCT_IDs"]:
        nct_to_pubmed[nct_id] = pubmed_id

# Sample all_disease DataFrame
# all_disease = pd.DataFrame({
#     "NCT ID": ["NCT06088706", "NCT02871856", "NCT12345678", "NCT05622630"]
# })

# Map NCT ID to PubMed ID
all_disease["Associated Article ID"] = all_disease["NCT ID"].map(nct_to_pubmed).fillna("")

# Create binary column indicating association
all_disease["Associated Article?"] = all_disease["Associated Article ID"].apply(lambda x: "YES" if x else "NO")

# Display result
print(all_disease)


           NCT ID  Acronym           Overall Status  Start Date  \
0     NCT02695966     EVIS                COMPLETED     2015-05   
1     NCT03882866  Unknown                  UNKNOWN  2019-03-28   
2     NCT04572165  Unknown  ENROLLING_BY_INVITATION  2021-01-26   
3     NCT03190265  Unknown                COMPLETED  2017-12-14   
4     NCT02822066  Unknown                  UNKNOWN     2016-04   
...           ...      ...                      ...         ...   
3631  NCT00592553  Unknown                COMPLETED  2008-02-29   
3632  NCT00102453  Unknown                COMPLETED     2002-03   
3633  NCT01350154  Unknown                COMPLETED     2011-11   
3634  NCT00016653  Unknown                COMPLETED     2000-06   
3635  NCT02653833  Unknown               TERMINATED  2017-11-01   

                                             Conditions  \
0                                     Pancreatic Cancer   
1                                     Pancreatic Cancer   
2                 

In [9]:
all_disease.head()

c=0
for disease in all_disease:
    print(disease)
    if disease['Associated Article?'] == 'YES':
        c=c+1
print(c)

NCT ID


TypeError: string indices must be integers, not 'str'

In [10]:
all_disease.head(5)

,NCT ID,Acronym,Overall Status,Start Date,Conditions,Interventions,Locations,Primary Completion Date,Study First Post Date,Last Update Post Date,Study Type,Phases,Sponsor,Sponsor Type,Disease,category,Associated Article ID,Associated Article?
0,NCT02695966,EVIS,COMPLETED,2015-05,Pancreatic Cancer,Whipples Surgery,Cambridge - United Kingdom,2018-07,2016-03-02,2019-07-16,OBSERVATIONAL,Not Available,Cambridge University Hospitals NHS Foundation ...,OTHER,pancreatic_cancer,all_disease_procesed.csv,,NO
1,NCT03882866,Unknown,UNKNOWN,2019-03-28,Pancreatic Cancer,"3D-printed non-coplanar template, 3D-printed c...","Hefei - China, Beijing - China, Guangzhou - Ch...",2021-06-30,2019-03-20,2020-02-06,INTERVENTIONAL,NaN,Ruijin Hospital,OTHER,pancreatic_cancer,all_disease_procesed.csv,,NO
2,NCT04572165,Unknown,ENROLLING_BY_INVITATION,2021-01-26,"Diabetes Mellitus, Type 2","Ozempic®, Active Comparator, Rybelsus®",Odense - Denmark,2024-12-31,2020-10-01,2024-01-02,OBSERVATIONAL,Not Available,Novo Nordisk A/S,INDUSTRY,pancreatic_cancer,all_disease_procesed.csv,,NO
3,NCT03190265,Unknown,COMPLETED,2017-12-14,Pancreatic Cancer,"Cyclophosphamide, Nivolumab, Ipilimumab, GVAX ...",Baltimore - United States,2022-08-03,2017-06-16,2024-08-07,INTERVENTIONAL,PHASE2,Sidney Kimmel Comprehensive Cancer Center at J...,OTHER,pancreatic_cancer,all_disease_procesed.csv,,NO
4,NCT02822066,Unknown,UNKNOWN,2016-04,Refractory Neoplasms,Electroporation,Hangzhou - China,2018-06,2016-07-04,2016-07-04,INTERVENTIONAL,PHASE2,First Affiliated Hospital of Zhejiang University,OTHER,pancreatic_cancer,all_disease_procesed.csv,,NO


In [19]:
print("Count of all diseases:", len(all_disease))
print("Number of diseases with associated articles:", all_disease['Associated Article?'].value_counts()['YES'])
print("Number of diseases not having any associated articles:", all_disease['Associated Article?'].value_counts()['NO'])


Count of all diseases: 3636
Number of diseases with associated articles: 422
Number of diseases not having any associated articles: 3214


In [ ]:
diseases_with_associated_articles = all_disease[all_disease['Associated Article?'] == 'YES']
diseases_without_associated_articles = all_disease[all_disease['Associated Article?'] == 'NO']
print(len(diseases_with_associated_articles))
print(len(diseases_without_associated_articles))




422
3214
